In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Author : Lavi Nigam, ML Engineering @ Google 
# Linkedin: https://www.linkedin.com/in/lavinigam/ 

<table align="left">

  <td>
    <a href="https://colab.research.google.com/https://github.com/lavinigam-gcp/BQML_Hackathon/blob/main/LTV_Prediction/BQ_Hackathon_StarterNotebook.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/lavinigam-gcp/BQML_Hackathon/blob/main/LTV_Prediction/BQ_Hackathon_StarterNotebook.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
</table>

Before starting the notebook, make sure you have two things: 

1) Acknowledging that you finished the pre-requisite [here](https://machinehack.com/hackathons/google_cloud_bigquery_ltv_prediction_challenge/overview?prerequisites=true)

2) Project ID from Google Cloud. 

Do not proceed further wihout these steps. 

Set your project ID here.

In [19]:
PROJECT_ID = ""
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "PUT_YOUR_PROJECT_ID_HERE"  # @param {type:"string"}

PROJECT_ID

'hackathon-373108'

### Authenticate your Google Cloud account

***Make sure you are logged out of all google (gmail) accounts except the one that you created for this Hackathon. We recommend using a separate browser, or make sure you have open Colab using the hackathon newly created Gmail account.***

In [13]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [14]:
# Importing some important libraries that will be used during the notebook
import pandas as pd
import plotly.express as px
from google.cloud import bigquery

In [20]:
#Client manages connections to the BigQuery API and helps
#bundle configuration (project, credentials) needed for API requests.
client = bigquery.Client(PROJECT_ID)

# to make sure all columns are displayed while working with dataframe
pd.set_option('display.max_columns', None)

## Assumptions

## Exploratory Data Analysis (EDA)

You can start by defining some essential variables that can change according to your data. It is always better to consider the most recent records from your data as features. For this purpose, you can set the START_DATE and END_DATE based on your data recency.

In this case, the date range is set for 3 months.

In [21]:
PROJECT_ID_DATA = "mh-hackathon"
DATASET_ID_DATA = "ga4_data" 
TABLE_ID_TRAIN  = "ga4_train"
TABLE_ID_TEST  = "ga4_test"
START_DATE = "20201101"
END_DATE = "20210131"

You can start the data exploration by returning the first five rows of data.
The data has multiple event tables for each day. So, all the tables (events) could be queried by using events* as the wildcard.

[GA4 Data Export Schema](https://support.google.com/analytics/answer/7029846#zippy=)

Note: BigQuery export, by default, are [date sharded tables](https://cloud.google.com/bigquery/docs/partitioned-tables#dt_partition_shard)

In [22]:
query = f"""
SELECT
  *
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`
LIMIT
  5
"""
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data.head()

,user_pseudo_id,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,privacy_info,user_properties,user_first_touch_timestamp,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,ltv
0,52407464.0313773688,20201203,1607033908954509,page_view,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,3102692248,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605035923000488,"{'category': 'mobile', 'mobile_brand_name': 'H...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],0.779221
1,52407464.0313773688,20201203,1607032477375233,session_start,"[{'key': 'ga_session_number', 'value': {'strin...",<NA>,NaN,6874157865,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605035923000488,"{'category': 'mobile', 'mobile_brand_name': 'H...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],0.779221
2,84760672.8640407660,20201229,1609202428116997,user_engagement,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,-1415807169,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1609202350156023,"{'category': 'mobile', 'mobile_brand_name': '<...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],68.918501
3,29934948.8545379469,20201208,1607391347326167,first_visit,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,-7005271747,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1607391347326167,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '<Other>', 'name': '<Other>', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],216.064620
4,8370439.6464221452,20201229,1609268001007966,session_start,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,-5508872159,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1609268001007966,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': 'referral', 'name': '(referral)', '...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],349.411800


In [23]:
query = f"""
SELECT
  *
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TEST}`
LIMIT
  5
"""
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data.head()

,user_pseudo_id,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,privacy_info,user_properties,user_first_touch_timestamp,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items
0,1000823.8498711409,20210118,1610938668079160,page_view,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,-9796371683,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610938663062961,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Americas', 'sub_continent': 'So...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
1,1000823.8498711409,20210118,1610938740311812,user_engagement,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,-2370677642,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610938663062961,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Americas', 'sub_continent': 'So...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
2,1000823.8498711409,20210118,1610938663062961,session_start,"[{'key': 'ga_session_number', 'value': {'strin...",<NA>,NaN,9869243676,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610938663062961,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Americas', 'sub_continent': 'So...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
3,1000985.4712566084,20210114,1610603132099178,session_start,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,2236704977,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610603132099178,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
4,1000985.4712566084,20210114,1610603132099178,page_view,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,2236704977,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610603132099178,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]


The first five rows of data can help you understand the tables' composite structure of data types. For example, you can see numerical, categorical, Arrays, and Struct as data types. Using this information, later, you will be able to write specific `UNNEST` queries for [Arrays](https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays#query_structs_in_an_array) & [Struct](https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays#querying_array-type_fields_in_a_struct).

By looking at some columns, you can also identify a few essential features like event_date, event_name, user_ltv, device, geo, traffic_source, platform, and items. However, as discussed earlier, you still are not aware of their value distribution, availability, and data types.

You can check the data types of each column using [INFORMATION_SCHEMA](https://cloud.google.com/bigquery/docs/information-schema-tables) table. It can give you detailed metadata of your columns.

In [ ]:
query = f"""
SELECT
  DISTINCT(column_name),
  data_type
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.INFORMATION_SCHEMA.COLUMNS`
"""

query_job = client.query(query)
predict_data = query_job.to_dataframe()
predict_data

,column_name,data_type
0,user_pseudo_id,STRING
1,event_date,STRING
2,event_timestamp,INT64
3,event_name,STRING
4,event_params,"ARRAY<STRUCT<key STRING, value STRUCT<string_v..."
5,event_previous_timestamp,INT64
6,event_value_in_usd,FLOAT64
7,event_bundle_sequence_id,INT64
8,event_server_timestamp_offset,INT64
9,user_id,STRING


You can start by understanding overall data by getting a quick summary of the data, namely - total events  (event_count), total users (user_count), total days in the data (day_count), and total registered users of the platform (registered_user_id).
This can help you get a sense of the scale of data.

In [9]:
query = f"""

SELECT
  COUNT(*) AS event_count,
  COUNT(DISTINCT user_pseudo_id) AS user_count,
  COUNT(DISTINCT event_date) AS day_count,
  COUNT(DISTINCT user_id) AS registered_user_id
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`
"""
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data

,event_count,user_count,day_count,registered_user_id
0,3859763,243394,92,0


In [10]:
query = f"""

SELECT
  COUNT(*) AS event_count,
  COUNT(DISTINCT user_pseudo_id) AS user_count,
  COUNT(DISTINCT event_date) AS day_count,
  COUNT(DISTINCT user_id) AS registered_user_id
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TEST}`
"""
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data

,event_count,user_count,day_count,registered_user_id
0,435821,26760,92,0


As you can observe, there are roughly 4 million events with close to 270,000 users, stretched along 92 days of activity on the platform.

There are no registered users data in the table. The user_pseudo_id is not a "user_id"; it is an client ID (cookie ID) for the user. This means that a single user can be represented as multiple pseudo_id in the data.

For simplicity, we will assume that all user_pseudo_id are unique and represent a single user.

If your data has 'user_id', use that directly, or else you can go ahead and use 'user_psuudo_id'.


### How to access Nested Data



---
Now, you can start by looking into `event_name` distribution.

event_name is a significant column in this dataset. It contains all the events triggered as users interact with the Google Merchandise Store like page_view, scroll (scrolling the page), view_item (viewing specific item), etc. You can refer [here](https://developers.google.com/analytics/devguides/collection/ga4/reference/events) for a more detailed meaning of each event_name.


In [11]:
query = f"""
SELECT
  event_name,
  COUNT(*) as row_count
FROM
   `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`
GROUP BY
  event_name
ORDER BY
  row_count DESC
"""
query_job = client.query(query)
result_df = query_job.to_dataframe()
fig = px.bar(result_df, x="row_count", y="event_name",  title="Event Name Frequency Distribution")
fig.show()

#### Accessing Array Struct

In [ ]:
query = f"""
SELECT
  DISTINCT(ep.key) AS event_param_key,
  COUNT(*) AS count
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`,
  UNNEST (event_params) AS ep
WHERE
  event_name = 'page_view'
GROUP BY
  ep.key
ORDER BY
  count DESC
"""
# print(query)
query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df

,event_param_key,count
0,page_location,1213143
1,ga_session_id,1213143
2,session_engaged,1213143
3,ga_session_number,1213143
4,debug_mode,1213077
5,page_title,1207539
6,engaged_session_event,1174367
7,all_data,1012876
8,clean_event,1012803
9,page_referrer,891451


In [ ]:
query = f"""
SELECT
  event_date,
  event_timestamp,
  event_name,
  user_pseudo_id,
  ep.key,
  ep.value
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`,
  UNNEST (event_params) AS ep
WHERE
  ep.key = 'page_title'
LIMIT
  10
  """
query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()

,event_date,event_timestamp,event_name,user_pseudo_id,key,value
0,20201213,1607851276376856,scroll,9179845131.5310230282,page_title,"{'string_value': 'Home', 'int_value': None, 'f..."
1,20201130,1606735459224193,view_item,11341074.5770059189,page_title,"{'string_value': 'Super G Unisex Joggers', 'in..."
2,20201225,1608871734244146,user_engagement,30091685.2976941851,page_title,{'string_value': 'YouTube | Shop by Brand | Go...
3,20201217,1608223512123667,page_view,83552580.7078798010,page_title,"{'string_value': 'Home', 'int_value': None, 'f..."
4,20201122,1606018175266973,user_engagement,43903452.9866415063,page_title,"{'string_value': 'Home', 'int_value': None, 'f..."


In [ ]:
query = f"""
SELECT
  event_date,
  event_timestamp,
  event_name,
  user_pseudo_id,
  ep.key,
  ep.value.string_value
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`,
  UNNEST (event_params) AS ep
WHERE
  ep.key = 'page_title'
LIMIT
  10
  """
query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()

,event_date,event_timestamp,event_name,user_pseudo_id,key,string_value
0,20210112,1610492841741927,user_engagement,78087117.2563336476,page_title,YouTube | Shop by Brand | Google Merchandise S...
1,20201121,1605972878234126,view_item,8594243.4562718268,page_title,Google Mural Socks
2,20201215,1608012431036172,user_engagement,10351288.5764317808,page_title,The Google Merchandise Store - Log In
3,20201208,1607397552383492,user_engagement,30900021.3233786964,page_title,Home
4,20201103,1604372926299442,scroll,30900021.3233786964,page_title,Android Puzzlebot v2


#### Accessing Struct

In [ ]:
query = f"""
SELECT
  event_date,
  event_timestamp,
  event_name,
  user_pseudo_id,
  geo.continent,
  geo.sub_continent,
  geo.country,
  geo.region
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`
LIMIT
  10
  """
query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()

,event_date,event_timestamp,event_name,user_pseudo_id,continent,sub_continent,country,region
0,20210112,1610492841741927,user_engagement,78087117.2563336476,Asia,Western Asia,Qatar,Doha
1,20201121,1605972878234126,view_item,8594243.4562718268,Asia,Western Asia,Qatar,Doha
2,20201215,1608012431036172,user_engagement,10351288.5764317808,Asia,Western Asia,Qatar,Doha
3,20201208,1607397552383492,user_engagement,30900021.3233786964,Asia,Western Asia,Qatar,Doha
4,20201103,1604372926299442,scroll,30900021.3233786964,Asia,Western Asia,Qatar,Doha


## Feature Engineering

Before we create our features, its important to create a dataset, where our new feature table will be stored. 

In [12]:
# You can create the dataset through code.
DATASET_NAME = "ga4_ecomm_feature_set"

try:
  dataset = client.create_dataset(DATASET_NAME, timeout=30)  # Make an API request.
  print("Created dataset {}.{}".format(client.project, dataset.dataset_id))
except Exception as e:
  print(e)

Created dataset hackathon-373108.ga4_ecomm_feature_set


Lets create some features that we can levrage to create Kmeans Cluster. For this example, we will simply take browsing data of each user. 

In [13]:
feature_table="ga4_features_train" #table name

In [14]:
query = f"""
CREATE OR REPLACE TABLE
  {DATASET_NAME}.{feature_table} AS
SELECT
user_pseudo_id, AVG(ltv) as avg_ltv, 
MAX(CAST(format_date('%m',parse_date("%Y%m%d",event_date)) as INT64)) as month_of_the_year,
MAX(CAST(format_date('%U',parse_date("%Y%m%d",event_date)) as INT64)) as week_of_the_year,
MAX(CAST(format_date('%d',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_the_month,
MAX(CAST(format_date('%w',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_week,

FROM 
`{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`
group by user_pseudo_id
  """
query_job = client.query(query)

In [15]:
query = """

SELECT
  *
FROM
  `ga4_ecomm_feature_set.ga4_features_train`
LIMIT
5
"""

query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()

,user_pseudo_id,avg_ltv,month_of_the_year,week_of_the_year,day_of_the_month,day_of_week
0,1829763.3860332334,11.895388,1,0,1,5
1,7032351.9989245504,16.855559,1,0,1,5
2,6772624.5257726160,16.362517,1,0,1,5
3,5807457.8598720960,9.610730,1,0,1,5
4,19116356.3050415718,212.763396,1,0,1,5


In [16]:
model_name = "customer_ltv_model"


linear_regression_query = f"""
CREATE OR REPLACE MODEL
  `{DATASET_NAME}.{model_name}` 
  OPTIONS (
      MODEL_TYPE = 'LINEAR_REG',
    L1_REG = 0.1,
    L2_REG = 1.0,
    EARLY_STOP = FALSE,
    MAX_ITERATIONS = 10,
    DATA_SPLIT_METHOD = 'NO_SPLIT',
    CATEGORY_ENCODING_METHOD = 'DUMMY_ENCODING',
    ENABLE_GLOBAL_EXPLAIN = TRUE,
    INPUT_LABEL_COLS = ['avg_ltv']
   ) 
AS
SELECT
  * except (user_pseudo_id)
FROM
  `{PROJECT_ID}.{DATASET_NAME}.{feature_table}`
WHERE
  avg_ltv IS NOT NULL
"""
print(linear_regression_query)
# query_job = client.query(linear_regression_query)
#RUN THIS ON BQ CONSOLE 


CREATE OR REPLACE MODEL
  `ga4_ecomm_feature_set.customer_ltv_model` 
  OPTIONS (
      MODEL_TYPE = 'LINEAR_REG',
    L1_REG = 0.1,
    L2_REG = 1.0,
    EARLY_STOP = FALSE,
    MAX_ITERATIONS = 10,
    DATA_SPLIT_METHOD = 'NO_SPLIT',
    CATEGORY_ENCODING_METHOD = 'DUMMY_ENCODING',
    ENABLE_GLOBAL_EXPLAIN = TRUE,
    INPUT_LABEL_COLS = ['avg_ltv']
   ) 
AS
SELECT
  * except (user_pseudo_id)
FROM
  `hackathon-373108.ga4_ecomm_feature_set.ga4_features_train`
WHERE
  avg_ltv IS NOT NULL



In [17]:
ml_evaluate_query = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL `{DATASET_NAME}.{model_name}`,
    (
    SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_NAME}.{feature_table}`
    WHERE
      avg_ltv IS NOT NULL))
"""
query_job = client.query(ml_evaluate_query)
ml_info_df = query_job.to_dataframe()
ml_info_df

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,66.389262,10816.568285,2.594297,49.733899,0.000025,0.000025


## Prediction on Test & Submission

In [19]:
# Creating features from test set 
feature_table_test ="ga4_features_test" #table name
query = f"""
CREATE OR REPLACE TABLE
  {DATASET_NAME}.{feature_table_test} AS
SELECT
user_pseudo_id, 
MAX(CAST(format_date('%m',parse_date("%Y%m%d",event_date)) as INT64)) as month_of_the_year,
MAX(CAST(format_date('%U',parse_date("%Y%m%d",event_date)) as INT64)) as week_of_the_year,
MAX(CAST(format_date('%d',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_the_month,
MAX(CAST(format_date('%w',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_week,

FROM 
`{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TEST}`
group by user_pseudo_id
  """
# print(query)
query_job = client.query(query)

In [20]:
query = """

SELECT
  *
FROM
  `ga4_ecomm_feature_set.ga4_features_test`
LIMIT
5
"""

query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()

,user_pseudo_id,month_of_the_year,week_of_the_year,day_of_the_month,day_of_week
0,7792363.8829018370,1,0,1,5
1,55303209.2930579791,1,0,1,5
2,44018444.8000399566,1,0,1,5
3,90145762.6807557994,1,0,1,5
4,4015519.0757720765,1,0,1,5


In [21]:
prediction_data_table_name = "model_prediction_ltv_test"
query = f"""
CREATE OR REPLACE TABLE
  {DATASET_NAME}.{prediction_data_table_name} AS
SELECT
  *
FROM
  ML.PREDICT(MODEL `{DATASET_NAME}.{model_name}`,
    (
    SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_NAME}.ga4_features_test`
    ))
"""
print(query)
query_job = client.query(query)


CREATE OR REPLACE TABLE
  ga4_ecomm_feature_set.model_prediction_ltv_test AS
SELECT
  *
FROM
  ML.PREDICT(MODEL `ga4_ecomm_feature_set.customer_ltv_model`,
    (
    SELECT
      *
    FROM
      `hackathon-373108.ga4_ecomm_feature_set.ga4_features_test`
    ))



In [24]:
query = f"""
SELECT
  *
FROM
  {PROJECT_ID}.{DATASET_NAME}.{prediction_data_table_name}
LIMIT 5
"""
# print(query)
query_job = client.query(query)
predict_data = query_job.to_dataframe()
predict_data.head()

,predicted_avg_ltv,user_pseudo_id,month_of_the_year,week_of_the_year,day_of_the_month,day_of_week
0,65.697627,6899201.8484270306,1,0,1,5
1,65.697627,2044855.8624761839,1,0,1,5
2,65.697627,11392224.5732544025,1,0,1,5
3,65.697627,4234864.4349517309,1,0,1,5
4,65.697627,58696560.6840710911,1,0,1,5


In [25]:
query = f"""
SELECT
  user_pseudo_id,predicted_avg_ltv
FROM
  {PROJECT_ID}.{DATASET_NAME}.{prediction_data_table_name}
"""
# print(query)
query_job = client.query(query)
submission_data = query_job.to_dataframe()
submission_data.head()

,user_pseudo_id,predicted_avg_ltv
0,6899201.8484270306,65.697627
1,2044855.8624761839,65.697627
2,11392224.5732544025,65.697627
3,4234864.4349517309,65.697627
4,58696560.6840710911,65.697627


In [26]:
## Create Submission 
submission = submission_data[['user_pseudo_id',"predicted_avg_ltv"]]
submission.to_csv("submission.csv",index=False)
submission.head()

,user_pseudo_id,predicted_avg_ltv
0,6899201.8484270306,65.697627
1,2044855.8624761839,65.697627
2,11392224.5732544025,65.697627
3,4234864.4349517309,65.697627
4,58696560.6840710911,65.697627


In [27]:
#make sure before submission that your shape is (26760, 2). Do not submit unless its of same shape. 
submission.shape

(26760, 2)